In [4]:
#Intall Delta Lake
!pip3 install delta-spark==2.2.0

Defaulting to user installation because normal site-packages is not writeable
  Using cached delta_spark-2.2.0-py3-none-any.whl (20 kB)


In [6]:
#Import Libraries
import pyspark
from delta import *

In [7]:
#Config SparkSession
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/vk/Library/Python/3.9/lib/python/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/vk/.ivy2/cache
The jars for the packages stored in: /Users/vk/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fa415653-7e2a-4044-a945-32fd09310550;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-core_2.12/2.2.0/delta-core_2.12-2.2.0.jar ...
	[SUCCESSFUL ] io.delta#delta-core_2.12;2.2.0!delta-core_2.12.jar (6460ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/2.2.0/delta-storage-2.2.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;2.2.0!delta-storage.jar (686ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.8/antlr4-runtime-4.8.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.8!antlr4-runtime.jar (964ms)
:: resolution report :: resolve 7224ms :: artifacts dl 8117m

23/04/04 21:31:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [8]:
#Create a Delta Lake Table from a DataFrame

columns = ["Name", "Age"]
data = [("ABC",25),("xyz",27),("zaq",31)]
rdd=spark.sparkContext.parallelize(data)
df=rdd.toDF(columns)

In [9]:
df.show()

+----+---+
|Name|Age|
+----+---+
| ABC| 25|
| xyz| 27|
| zaq| 31|
+----+---+



In [10]:
df.write.format("delta").saveAsTable("MyFirstDeltaTable")

23/04/04 21:34:22 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [11]:
#run a command to confirm that the table is in fact a Delta Lake table
DeltaTable.isDeltaTable(spark,"spark-warehouse/myfirstdeltatable/")

True

In [13]:
#Describe the table and check the version and timestamp
spark.sql("DESCRIBE HISTORY myfirstdeltatable").select("version","timestamp").show(truncate=False)

+-------+-----------------------+
|version|timestamp              |
+-------+-----------------------+
|0      |2023-04-04 21:34:22.332|
+-------+-----------------------+



In [14]:
#Show the table
spark.table("myfirstdeltatable").show()

+----+---+
|Name|Age|
+----+---+
| zaq| 31|
| xyz| 27|
| ABC| 25|
+----+---+




# Create a Empty Delta Lake table with the PySpark API


In [15]:
from pyspark.sql.types import *

dt1 = (DeltaTable.create(spark).tableName("DeltaTableUsingPyAPI")
       .addColumn("Name",dataType="STRING",nullable=False).addColumn("Age",dataType="INT")
       .execute())


In [17]:
dt1

In [16]:
spark.table("DeltaTableUsingPyAPI").show()

+----+---+
|Name|Age|
+----+---+
+----+---+



In [18]:
spark.sql("DESCRIBE HISTORY DeltaTableUsingPyAPI").select("version","timestamp").show()

+-------+--------------------+
|version|           timestamp|
+-------+--------------------+
|      0|2023-04-04 21:53:...|
+-------+--------------------+



# Create a Delta Lake table with SQL

In [19]:
spark.sql("""
          CREATE TABLE DeltaTableUsingSQL (Name STRING , AGE INTEGER) USING delta
          """)

DataFrame[]

In [20]:
spark.sql("""
          INSERT INTO DeltaTableUsingSQL VALUES
          ('ABC',25),
          ('XYZ',28)
          """)

DataFrame[]

In [22]:
spark.sql("DESCRIBE DETAIL DeltaTableUsingSQL").select("format").show()

+------+
|format|
+------+
| delta|
+------+

